In [1]:
import pandas as pd
import numpy as np
import pymysql

<h1 align='center'>Implementing Data to MySQL Database</h1>

<h2>Overview:</h2><br>

This Jupyter notebook contains Python code for implementing data to a MySQL database. The code includes functions for establishing a connection to the database, creating tables, and inserting data into the tables. The notebook is specific to a tourism analysis project and includes tables for storing data on countries, municipalities and accommodation. The code reads in cleaned data from CSV files and inserts it into the appropriate tables.

In [2]:
def initiate_connection():
    try:
        connection = pymysql.connect(
            host='127.0.0.1',
            port=3306,
            user='root',
            passwd='########',
            db='tourism_analysis_mne',
        )
        print('Connection was succesfull!')
    except Exception as e:
        print('Connection was not succesfull!')
        connection = None
    return connection

In [3]:
connection = initiate_connection()
cursor=connection.cursor()

Connection was succesfull!


<h3>1. Implementing the country and tourists_by_country tables and populating them with data in the database</h3>

In [4]:
def create_tables(sql_query):
    cursor.execute(sql_query)
    connection.commit()

In [5]:
sql_query_country = f'''CREATE TABLE IF NOT EXISTS country(
        id INT PRIMARY KEY AUTO_INCREMENT,
        name VARCHAR(255) UNIQUE);'''
create_tables(sql_query_country)

In [6]:
serbian_to_english = {'Albanija': 'Albania',
                     'Austrija': 'Austria',
                     'Belgija': 'Belgium',
                     'Bjelorusija': 'Belarus',
                     'Bosna i Hercegovina': 'Bosnia and Herzegovina',
                     'Bugarska': 'Bulgaria',
                     'Češka Republika': 'Czech Republic',
                     'Danska': 'Denmark',
                     'Estonija': 'Estonia',
                     'Finska': 'Finland',
                     'Francuska': 'France',
                     'Grčka': 'Greece',
                     'Holandija': 'Netherlands',
                     'Hrvatska': 'Croatia',
                     'Irska': 'Ireland',
                     'Island': 'Iceland',
                     'Italija': 'Italy',
                     'Kosovo': 'Kosovo',
                     'Letonija': 'Latvia',
                     'Litvanija': 'Lithuania',
                     'Luksemburg': 'Luxembourg',
                     'Mađarska': 'Hungary',
                     'Republika Sjeverna Makedonija': 'North Macedonia',
                     'Norveška': 'Norway',
                     'Njemačka': 'Germany',
                     'Poljska': 'Poland',
                     'Portugalija': 'Portugal',
                     'Rumunija': 'Romania',
                     'Rusija': 'Russia',
                     'Slovačka': 'Slovakia',
                     'Slovenija': 'Slovenia',
                     'Srbija': 'Serbia',
                     'Švajcarska': 'Switzerland',
                     'Švedska': 'Sweden',
                     'Španija': 'Spain',
                     'Turska': 'Turkey',
                     'Ukrajina': 'Ukraine',
                     'Ujedinjeno Kraljevstvo': 'United Kingdom',
                     'Ostale evropske zemlje': 'Other European countries',
                     'Australija': 'Australia',
                     'Izrael': 'Israel',
                     'Japan': 'Japan',
                     'Kanada': 'Canada',
                     'Novi Zeland': 'New Zealand',
                     'Sjedinjene Američke Države': 'United States of America',
                     'Ostale vanevropske zemlje': 'Other non-European countries',
                     'Kipar': 'Cyprus',
                     'Malta': 'Malta',
                     'Južna Afrika': 'South Africa',
                     'Ostale afričke zemlje': 'Other African countries',
                     'Ostale zemlje Sjeverne Amerike': 'Other North American countries',
                     'Argentina': 'Argentina',
                     'Brazil': 'Brazil',
                     'Čile': 'Chile',
                     'Ostale zemlje Južne i Srednje Amerike': 'Other South and Central American countries',
                     'Kina': 'China',
                     'Indija': 'India',
                     'Azerbejdžan': 'Azerbaijan',
                     'Ostale azijske zemlje': 'Other Asian countries',
                     'Koreja, Republika (Južna Koreja)': 'South Korea',
                     'Ostale azijske zemlje': 'Other Asian Countries',
                     'Ostale zemlje Okeanije': 'Other Oceanian Countries',
                     'Ujedinjeni Arapski Emirati': 'United Arab Emirates'
                    

} 

In [7]:
clean_country_name = {'Češka': 'Češka Republika','Makedonija': 'Republika Sjeverna Makedonija', 'Ruska Federacija': 'Rusija',
                     'Srbija (bez Kosova)': 'Srbija','Švajcarska uključujući Lihtenštajn': 'Švajcarska',
                     'SAD': 'Sjedinjene Američke Države','Velika Britanija': 'Ujedinjeno Kraljevstvo', 'Švajcarska (uključujući Lihtenštajn)': 'Švajcarska',
                     'Kina (uključujući Hong Kong)': 'Kina', 'Koreja, Republika': 'Koreja, Republika (Južna Koreja)'}

In [8]:
def clean_country(country_name):
    if country_name in clean_country_name.keys():
        country_name = clean_country_name[country_name]
        return serbian_to_english[country_name]
    else:
        return serbian_to_english[country_name]

In [13]:
def insert_data_to_country():
    help_list = []
    for year in range(2012, 2023):
        df = pd.read_csv(fr'C:\Users\Ivan\Desktop\tourism_analysis\data\cleaned_data\final_data_country\tourists_by_country_{year}.csv')
        for index, row in df.iterrows():
            country_name = row['zemlja_porijekla'].strip()
            country_name = clean_country(country_name)
            if country_name not in help_list:
                help_list.append(country_name)
                try:
                    sql_insert = f"INSERT INTO country(`name`) VALUES('{country_name}');"
                    cursor.execute(sql_insert)
                    connection.commit()
                except Exception as e:
                    continue
insert_data_to_country()

In [10]:
sql_create_table = f'''CREATE TABLE tourists_by_country(
        id INT PRIMARY KEY AUTO_INCREMENT,
        year INT,
        country_id INT,
        arrivals INT,
        nights INT,
        FOREIGN KEY (country_id) REFERENCES country(id))'''
create_tables(sql_create_table)

In [11]:
def insert_data_into_tourists_country():
    for year in range(2012, 2023):
        df = pd.read_csv(fr'C:\Users\Ivan\Desktop\tourism_analysis\data\cleaned_data\final_data_country\tourists_by_country_{year}.csv')
        for index, row in df.iterrows():
            country_name = row['zemlja_porijekla'].strip()
            country_name = clean_country(country_name)
            try:
                sql_insert = f'''INSERT INTO tourists_by_country(`year`, `country_id`, `arrivals`, `nights`)
                VALUES({year},
                (SELECT id FROM country where name = "{country_name}"),
                {row["dolasci"]},
                {row["nocenja"]});'''
                cursor.execute(sql_insert)
                connection.commit()
            except Exception as e:
                print('Mistake has been made')
insert_data_into_tourists_country()

<h3>2. Implementing the municipality and tourists_by_municipality tables and populating them with data in the database</h3>

In [14]:
sql_query = f'''CREATE TABLE IF NOT EXISTS municipality (
                id INT PRIMARY KEY AUTO_INCREMENT,
                name VARCHAR(255) UNIQUE
);'''
create_tables(sql_query)

In [15]:
def insert_data_to_municipality():
    help_list = []
    for year in range(2012, 2023):
        df = pd.read_csv(fr'C:\Users\Ivan\Desktop\tourism_analysis\data\cleaned_data\final_data_municipality\tourists_by_municipality_{year}.csv')
        for index, row in df.iterrows():
            mun_name = row['opstina'].strip()
            if mun_name not in help_list:
                help_list.append(mun_name)
                try:
                    sql_insert = f"INSERT INTO municipality(`name`) VALUES('{mun_name}');"
                    cursor.execute(sql_insert)
                    connection.commit()
                except Exception as e:
                    continue
insert_data_to_municipality()

In [16]:
sql_query = f'''
    CREATE TABLE tourists_by_municipality (
        id INT PRIMARY KEY AUTO_INCREMENT,
        year INT,
        municipality_id INT,
        arrivals_foreign INT,
        arrivals_domestic INT,
        nights_foreign INT,
        nights_domestic INT,
        FOREIGN KEY (municipality_id) REFERENCES municipality(id)
);
'''
create_tables(sql_query)

In [17]:
def insert_data_into_tourists_mun():
    for year in range(2012, 2023):
        df = pd.read_csv(fr'C:\Users\Ivan\Desktop\tourism_analysis\data\cleaned_data\final_data_municipality\tourists_by_municipality_{year}.csv')
        for index, row in df.iterrows():
            mun_name = row['opstina'].strip()
            try:
                sql_insert = f'''INSERT INTO tourists_by_municipality(`year`, `municipality_id`, 
                                `arrivals_foreign`, `arrivals_domestic`, `nights_foreign`,`nights_domestic`)
                                VALUES(
                                        {year},
                                        (SELECT id FROM municipality where name = "{mun_name}"),
                                        {row["dolasci_stranih_turista"]},
                                        {row["dolasci_domacih_turista"]},
                                        {row["nocenje_stranih_turista"]},
                                        {row["nocenje_domacih_turista"]}
                                        );'''
                cursor.execute(sql_insert)
                connection.commit()
            except Exception as e:
                print(year, row)
insert_data_into_tourists_mun()

<h3>3. Implementing the accommodation and tourists_by_accommodation tables and populating them with data in the database</h3>

In [18]:
sql_query = f'''
    CREATE TABLE accommodation (
      id INT PRIMARY KEY AUTO_INCREMENT,
      type VARCHAR(255) UNIQUE
);
'''
create_tables(sql_query)

In [25]:
clean_accommodation = {'Apart hoteli': 'Apart hotel', 'Gostionice': 'Gostionica', 'Gostionica i pansion': 'Gostionica',
                      'Hoteli': 'Hotel', 'Kampovi': 'Kamp', 'Lječilišta': 'Lječilište', 'Moteli': 'Motel',
                      'Odmarališta': 'Odmaralište', 'Odmaralište i lječilište': 'Odmaralište',
                      'Pansioni': 'Pansion', 'Pansion i vila': 'Pansion', 'Prenoćišta (hosteli)': 'Hostel',
                      'Turistička naselja':'Turističko naselje'}

In [27]:
def clean_name(name):
    if name in clean_accommodation.keys():
        return clean_accommodation[name]
    else:
        return name
    

In [28]:
def insert_data_to_accomodation():
    help_list = []
    for year in range(2012, 2023):
        df = pd.read_csv(fr'C:\Users\Ivan\Desktop\tourism_analysis\data\cleaned_data\final_data_accomodation\tourists_by_accomodation_{year}.csv')
        for index, row in df.iterrows():
            acc_name = clean_name(row['tip_smjestaja'].strip())
            if acc_name not in help_list:
                help_list.append(acc_name)
                try:
                    sql_insert = f"INSERT INTO accommodation(`type`) VALUES('{acc_name}');"
                    cursor.execute(sql_insert)
                    connection.commit()
                except Exception as e:
                    print(row)
                    continue
insert_data_to_accomodation()

In [29]:
sql_query = f'''
  CREATE TABLE tourists_by_accommodation (
  id INT PRIMARY KEY AUTO_INCREMENT,
  year INT,
  accommodation_id INT,
  arrivals_foreign INT,
  arrivals_domestic INT,
  nights_foreign INT,
  nights_domestic INT,
  FOREIGN KEY (accommodation_id) REFERENCES accommodation(id)
);
'''
create_tables(sql_query)

In [30]:
def insert_data_to_tourists_accommodation():
    for year in range(2012, 2023):
        df = pd.read_csv(fr'C:\Users\Ivan\Desktop\tourism_analysis\data\cleaned_data\final_data_accomodation\tourists_by_accomodation_{year}.csv')
        for index, row in df.iterrows():
            acc_name = clean_name(row['tip_smjestaja'].strip())
            if year == 2015 or year == 2016:
                try:
                    sql_insert = f'''INSERT INTO tourists_by_accommodation(`year`, 
                                    `accommodation_id`, `arrivals_foreign`, `nights_foreign`) 
                                VALUES(
                                {year}, 
                                (SELECT id FROM accommodation where type="{acc_name}"),
                                {row["dolasci_turista"]},
                                {row["nocenje_turista"]}
                                );'''
                    cursor.execute(sql_insert)
                    connection.commit()
                except Exception as e:
                    print(year, row)
                continue
            else:
                try:
                    sql_insert = f'''INSERT INTO tourists_by_accommodation(`year`, 
                                    `accommodation_id`, `arrivals_foreign`, 
                                    `arrivals_domestic`, `nights_foreign`,`nights_domestic`) 
                                VALUES(
                                {year}, 
                                (SELECT id FROM accommodation where type="{acc_name}"),
                                {row["dolasci_stranih_turista"]},
                                {row["dolasci_domacih_turista"]},
                                {row["nocenje_stranih_turista"]},
                                {row["nocenje_domacih_turista"]}
                                );'''
                    cursor.execute(sql_insert)
                    connection.commit()
                except Exception as e:
                    print(year, row)
                    continue
insert_data_to_tourists_accommodation()

In [ ]:
connection.close()